In [17]:
ls

checkpoint/  data/  model/  README.md  requirements.txt  test.py  train.py


In [15]:
import torch
import torch.nn as nn
import random

from kobert_transformers import get_tokenizer
from model.classifier import KoBERTforSequenceClassfication

In [3]:
import torch
import torch.nn as nn

In [16]:
my_model = torch.load('../model/kobert/model_kobert_chatbot_60epoch.pt')

In [ ]:
def load_wellness_answer():
    root_path = "."
    category_path = f"{root_path}/data/wellness_dialog_category.txt"
    answer_path = f"{root_path}/data/wellness_dialog_answer.txt"

    c_f = open(category_path, 'r')
    a_f = open(answer_path, 'r')

    category_lines = c_f.readlines()
    answer_lines = a_f.readlines()

    category = {}
    answer = {}
    for line_num, line_data in enumerate(category_lines):
        data = line_data.split('    ')
        category[data[1][:-1]] = data[0]

    for line_num, line_data in enumerate(answer_lines):
        data = line_data.split('    ')
        keys = answer.keys()
        if (data[0] in keys):
            answer[data[0]] += [data[1][:-1]]
        else:
            answer[data[0]] = [data[1][:-1]]

    return category, answer


def kobert_input(tokenizer, str, device=None, max_seq_len=512):
    index_of_words = tokenizer.encode(str)
    token_type_ids = [0] * len(index_of_words)
    attention_mask = [1] * len(index_of_words)

    # Padding Length
    padding_length = max_seq_len - len(index_of_words)

    # Zero Padding
    index_of_words += [0] * padding_length
    token_type_ids += [0] * padding_length
    attention_mask += [0] * padding_length

    data = {
        'input_ids': torch.tensor([index_of_words]).to(device),
        'token_type_ids': torch.tensor([token_type_ids]).to(device),
        'attention_mask': torch.tensor([attention_mask]).to(device),
    }
    return data


if __name__ == "__main__":
    root_path = "."
    checkpoint_path = f"{root_path}/checkpoint"
    save_ckpt_path = f"{checkpoint_path}/kobert-wellness-text-classification.pth"

    # 답변과 카테고리 불러오기
    category, answer = load_wellness_answer()

    ctx = "cuda:8"
    device = torch.device(ctx)

    # 저장한 Checkpoint 불러오기
    ##checkpoint = torch.load(save_ckpt_path, map_location=device)

    ##model = KoBERTforSequenceClassfication()
    ##model.load_state_dict(checkpoint['model_state_dict'])

    model = my_model
    model.to(ctx)
    model.eval()

    tokenizer = get_tokenizer()

    while 1:
        sent = input('\nQuestion: ')  # '요즘 기분이 우울한 느낌이에요'
        data = kobert_input(tokenizer, sent, device, 512)

        if '종료' in sent:
            break

        output = model(**data)

        logit = output[0]
        softmax_logit = torch.softmax(logit, dim=-1)
        softmax_logit = softmax_logit.squeeze()

        max_index = torch.argmax(softmax_logit).item()
        max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

        answer_list = answer[category[str(max_index)]]
        answer_len = len(answer_list) - 1
        answer_index = random.randint(0, answer_len)
        print(f'Answer: {answer_list[answer_index]}, index: {max_index}, softmax_value: {max_index_value}')
        print('-' * 50)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.



Question: 안녕
Answer: 잠시 쉬어간다고 생각해보는 건 어때요?, index: 55, softmax_value: 0.7230838537216187
--------------------------------------------------

Question: 하루 잘 보냈어?
Answer: 아, 그래요?, index: 90, softmax_value: 0.833807110786438
--------------------------------------------------

Question: 피곤하네
Answer: 휴식을 취할 때가 된 것 아닐까요? 잠깐 쉬어가요~, index: 337, softmax_value: 0.8274337649345398
--------------------------------------------------

Question: 눈이 침침해
Answer: 귀마개나 안대 등 숙면을 도와주는 물품을 사용해보는 건 어떨까요?, index: 300, softmax_value: 0.11854656040668488
--------------------------------------------------

Question: 눈물이 나와 ㅠㅠ
Answer: 감수성이 풍부하다는 증거 같아요. 눈물이 많은 사람 중에 나쁜 사람은 없잖아요., index: 22, softmax_value: 0.9927871823310852
--------------------------------------------------
